# Data Analysis Content
1. Python Libraries
2. Data Content
3. Read and Analyse Data
4. Visualization
5. Predicting protein  using ML
6. Conclusion

# 1. Python Libraries

In [115]:
#Load packages
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib.pyplot import plot, show, savefig, xlim, figure, \
                ylim, legend, boxplot, setp, axes
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,OrdinalEncoder,LabelBinarizer
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
from sklearn.feature_selection import RFE,SelectFromModel
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
plt.style.use('Solarize_Light2')


# 2. Data Content
> Wrangle, prepare, cleanse the data
Having a protein structure provides a greater level of understanding of how a protein works, which can allow us to create hypotheses about how to affect it, control it, or modify it. For example, knowing a protein's structure could allow you to design site-directed mutations with the intent of changing function.

   Currently, the main techniques used to determine protein 3D structure are **X-ray** crystallography and nuclear magnetic resonance (**NMR**). In **X-ray** crystallography the protein is **crystallized** and then using X-ray diffraction the structure of protein is determined.


In [116]:
# Importing the first dataset
df = pd.read_csv ("../input/protein-data-set/pdb_data_no_dups.csv")
df.head(2)

> Columns: 
1. structureId
2. classification
3. experimentalTechnique
4. macromoleculeType
5. death_num, Deaths Number
6. residueCount
7. resolution
8. structureMolecularWeight
9. crystallizationMethod
10. crystallizationTempK
11. densityMatthews
12. densityPercentSol 
13. pdbxDetails 
14. phValue 
15. publicationYear

In [145]:
df.info()
# calculate duplicates
dups = df.duplicated()
# report if there are any duplicates
print(dups.any())
# list all duplicate rows

print("Duplicate Rows",df[dups])
#Cardinality 
df.nunique() # To determine the maximum and minimum number of variations in each column of the dataset
#Lets now check for null fields
import seaborn as sns
plt.figure(figsize=(6,6))
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')
df.isnull().sum()


In [118]:
# Importing the second dataset
df1 = pd.read_csv ("../input/protein-data-set/pdb_data_seq.csv")
df1.head()

In [119]:
df1.info()
# calculate duplicates
dups = df1.duplicated()
# report if there are any duplicates
print(dups.any())
# list all duplicate rows

print(" * Duplicate Rows *",df1[dups])
#Cardinality 
df1.nunique() # To determine the maximum and minimum number of variations in each column of the dataset
#Lets now check for null fields
#plt.figure(figsize=(12,10))
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')
df1.isnull().sum()

# 3. Read and Analyse Data

In [120]:
# Merging the two datasets
df2 = df.set_index('structureId').merge(df1.set_index('structureId'),on='structureId',how='left')
df2.head()

In [146]:
# Clearing the classification column.
df2["classification"]=df2["classification"].str.replace('\n','')
df2["classification"]=df2["classification"].str.replace('\r','')
df2["classification"]=df2["classification"].str.replace('-','')
df2["classification"]=df2["classification"].str.replace(',','')
df2["classification"]=df2["classification"].str.replace('/','')
df2["classification"]=df2["classification"].str.replace(' ','')
df2["classification"]
df2['classification'].unique()
# Clearing the experimentalTechnique column.
df2["experimentalTechnique"]=df2["experimentalTechnique"].str.replace('\n','')
df2["experimentalTechnique"]=df2["experimentalTechnique"].str.replace('\r','')
df2["experimentalTechnique"]=df2["experimentalTechnique"].str.replace('-','')
df2["experimentalTechnique"]=df2["experimentalTechnique"].str.replace(',','')
df2["experimentalTechnique"]=df2["experimentalTechnique"].str.replace('/','')
df2["experimentalTechnique"]=df2["experimentalTechnique"].str.replace(' ','')
df2["experimentalTechnique"]
#df2['experimentalTechnique'].unique()

In [122]:
df2.describe().T.style.bar(subset=['mean'], color='#205ff2')\
                            .background_gradient(subset=['std'], cmap='Reds')\
                            .background_gradient(subset=['50%'], cmap='coolwarm')

In [123]:
print(" The shape of the protein dataset is: " +str(df2.shape))
# So we have > 400,000 observations (rows) and 17 features (columns)
print(df.max())

In [124]:
# Data Cleaning¶
# Drop unwanted columns(not relevant for our model)
df3 = df2.drop(['publicationYear', 'chainId','macromoleculeType_x', 'macromoleculeType_y'], axis = 1)
df3.shape
# Data Cleaning - Summing NA values per column
print(df2.isnull().sum())
# Drop na observations
df_new=df3.dropna(how='any')
print(df_new.isnull().sum())
df_new.head(2)

> obseravation
The data has been cleared.

# 4. Visualization

In [125]:
df3.hist(bins=10, figsize=(16, 10))
plt.tight_layout()

4. Visualization

In [126]:
# Verifying the experimentalTechnique
plt.figure(figsize=(6,6))
sns.countplot(data=df3,x = class_data['experimentalTechnique'],palette='plasma')

In [127]:
# Verifying the residueCount_x
plt.figure(figsize=(8,10))
sns.countplot(data=df3,x = df3['residueCount_x'],palette='plasma')

In [128]:
# Verifying the phValue
df3.phValue = df3.phValue.round()
plt.figure(figsize=(8,8))
sns.countplot(data=df3,x = df3['phValue'],palette='plasma')

> Significance of pH

 The pH range where proteins work best is neutral (pH 7) plus one and minus one, and their best activity occurs under acidic conditions (pH 4-6)

In [129]:
#correlation matrix
corrmat = df3.corr()
f, ax = plt.subplots(figsize=(10, 7))
sns.heatmap(corrmat, annot=True, linewidths=.5, fmt= '.1f',ax=ax);

In [130]:
# Removing correlated features: residueCount and densityMatthews
df4 = df_new.drop(['residueCount_x', 'residueCount_y','densityMatthews'],axis=1)
# Making the new heatmap now
#sns.heatmap(df4.corr(),cmap='BuPu',fmt= '.1f',ax=ax)
df4.shape
sns.heatmap(df4.corr(),cmap='BuPu',fmt= '.1f',ax=ax)

> *  Using a countplote to draw meaningful conclusions from the counting of different values**

In [131]:
# Verifying the experimentalTechnique
#df3['classification'].nunique()
plt.figure(figsize=(8,6))
df4['classification'].value_counts()[:10].plot(kind='bar')

In [132]:
df_1 = df4['classification'].value_counts()[:10]
print(df_1)
# Selecting families with count > 15000
counts = df4.classification.value_counts()
class_data = np.asarray(counts[(counts > 15000)].index)
class_data = df4[df4.classification.isin(class_data)]
# Checking new dataframe for top protein family names
class_data.classification.value_counts()

In [133]:
# Checking the size of new dataframe (essentially down to 25% of original data)
class_data.shape

In [134]:
class_data.dtypes 
# Converting all the categorical features to numerical
cat_transformer = OrdinalEncoder()
cat_features = ['crystallizationMethod', 'experimentalTechnique','pdbxDetails','sequence', 'classification']
transformed_cat = cat_transformer.fit_transform(class_data[cat_features])
class_data[cat_features] = transformed_cat
class_data.head(5)

> EDA

In [135]:
# Boxplot for visualizing the resolution feature
plt.figure(figsize=(8,6))
sns.boxplot(x=class_data["classification"], y=class_data["resolution"], data=pd.melt(class_data))
plt.ylabel("Protein Resolution (in Angstrom)", fontsize = 12)
#plt.xlabel("Protein Family", fontsize = 12)
#plt.yticks([0, 1000000, 2000000, 3000000, 4000000, 5000000, 6000000, 7000000, 8000000], ["0", "1", "2", "3", "4", "5", "6", "7", "8"])
plt.xticks([0, 1, 2, 3], ["Hydrolase","Oxidoreductase", "Ribosome", "Transferase"], rotation = 45)
plt.show()


In [136]:
# Pie Chart for visualizing the top 4 most common protein crystallization methods
plt.figure(figsize=(12,10))
count = class_data.crystallizationMethod.value_counts(ascending = False)
top_count = np.asarray(count[(count > 1500)].index)
top_CM = class_data[class_data.crystallizationMethod.isin(top_count)]
fig = top_CM.groupby(['crystallizationMethod']).sum().plot(kind='pie', y = 'sequence' , autopct='%1.0f%%', labels = None)
plt.title("Four most common protein crystallization methods")
plt.legend(labels = ["Microbatch", "Vapor Diffusion", "Vapor Diffusion, Hanging Drop", "Vapor Diffusion, Sitting Drop"], loc = 1)


In [137]:
# Comparing the molecular weights of the top 4 protein families using a boxplot
plt.figure(figsize=(8,6))
ax = sns.boxplot(x=class_data["classification"], y=class_data["structureMolecularWeight"], data=pd.melt(class_data))
ax.set_yscale('log')
plt.ylabel("Molecular Weight (in Daltons)", fontsize = 12)
#plt.xlabel("Protein Family")
plt.xticks([0, 1, 2, 3], ["Hydrolase","Oxidoreductase", "Ribosome", "Transferase"], rotation = 45)
#plt.yticks([0, 1000000, 2000000, 3000000, 4000000, 5000000, 6000000, 7000000, 8000000], ["0", "1", "2", "3", "4", "5", "6", "7", "8"])
plt.show()

> Obseravation

 Higher molecular weight and more readily identifiable ribosomes

# 5. Protein categorization using KNN model

> Building KNN model w/ feature selection

 Manual feature selection based on EDA, intuition, and domain knowledge was performed on the ‘clean protein dataset’ to bring the number of the feature inputs down to 5 – structure molecular weight, pH, crystallization method, crystallization temperature, and sequence.



In [138]:
# Build a classification model for this new dataframe (top 3 protein families)
# Scale y ?
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = preprocessing.MinMaxScaler()
X = class_data.drop(['classification', 'experimentalTechnique','pdbxDetails', "resolution", "densityPercentSol"],axis=1) 
y = class_data.classification # Classes: 0-3
X_minmax = min_max_scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_minmax, y, test_size=0.2, random_state=42)# are these good options?
print(X.columns)


In [139]:
# Building model w/ K=5 since it is the most commomly used K value
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Assessing the initial knn model's performance
y_pred = knn.predict(X_test)
#score = metrics.accuracy_score(y_test, y_pred)
#print("knn model accuracy: %0.3f" % score)
plt.figure(figsize=(10,8))
plot_confusion_matrix(knn,X_test,y_test,cmap=plt.cm.Blues, values_format = '.5g', display_labels = ["Hydrolase","Oxidoreductase", "Ribosome", "Transferase"])
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.show()
print(classification_report(y_test, y_pred, target_names=["Hydrolase","Oxidoreductase", "Ribosome", "Transferase"]))


> Obseravation:


The average structure molecular weight of the Ribosome proteins is ~20x higher than that of the other families. Additionally, Ribosome is the only class of proteins that does not have any outliers in the molecular weight column. Therefore, it is easier for the model to correctly classify proteins belonging to the Ribosome family.

In [140]:
# Usig GridSearchCV to find the best k values
import time
knn = KNeighborsClassifier()
begin_knn = time.time()
params = {'n_neighbors': np.arange(1,25,1) }
clf = GridSearchCV(knn, params, cv=10)
clf.fit(X_train,y_train)
end_knn = time.time()
print(clf.best_score_)
# total time taken
print(f"Total runtime of the knn program is {end_knn - begin_knn} seconds")

In [141]:
# Find the best k parameter
print("The best k parameter is : " + str(clf.best_params_))
print("Accuracy: " + str(clf.best_estimator_.score(X_test, y_test)))

In [142]:
# Calculating error for K values between 1 and 25
error = []
for i in range(1, 25):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    error.append(np.mean(pred_i != y_test))
plt.figure(figsize=(10,8))
plt.plot(range(1, 25), error, color='red', linestyle='dashed', marker='o',
         markerfacecolor='blue', markersize=10)
plt.title('ERROR RATE K VALUE',fontsize = 14 )
plt.xlabel('K Value', fontsize = 14 )
plt.ylabel('Mean Error', fontsize = 14 )


In [143]:
# Based on the above graph, K=1 gives us the lowest mean error (5 features finally!)
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_train, y_train)
plot_confusion_matrix(knn1,X_test,y_test,cmap=plt.cm.Blues,values_format = '.5g', display_labels = ["Hydrolase","Oxidoreductase", "Ribosome", "Transferase"])
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.show()


# Assessing the initial knn model's performance
y_pred = knn1.predict(X_test)
print(classification_report(y_test, y_pred, target_names=["Hydrolase","Oxidoreductase", "Ribosome", "Transferase"]))


# 6. Conclusion

In this project, a precise protein classification prediction system based on the physiochemical features of proteins is presented. For predicting the families (and subsequently the functions) of four different types of proteins, I was able to find a small and practical feature subset in this study. My KNN model performed better with K hyperparameter adjustment as the classification accuracy increased from 85% to 95% for K = 1 and n features = 5.